In [1]:
!pip install obspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# read, format and classify data from original mseed files

Ong, Giani, Nielsen

Modified for multiple stations by Dewsnap

**mount Google Drive as a disk to access files and data**



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd "/content/drive/MyDrive/DISS_FOLDER/New_Data"

/content/drive/MyDrive/DISS_FOLDER/New_Data


### **INITIALISATION**

In [4]:
choice = 'new2' # old=norm on all 3 components together. new2=no norm. new=norm on individual components,


years = [2014,2015,2016,2017,2018,2019,2020,2021,2022] #add more when more folders are finished
#2013 has no events with full data
#INCN is offline for most of 2012-2013 (alongside other stations missing entries/axes)
#2012 all entries have anomalies.
numStations = 3

stationNames = ["MAJO.IU","ERM.II","INCN.IU"]

for i in range(numStations):
  print("Taking events from station "+stationNames[i])


Taking events from station MAJO.IU
Taking events from station ERM.II
Taking events from station INCN.IU


In [5]:


def getOutputs():

  dataString = """#myId|#EventID | Time | Latitude | Longitude | Depth/km | Author | Catalog | Contributor | ContributorID | MagType | Magnitude | MagAuthor | EventLocationName
  #myId|#EventID | Time | Latitude | Longitude | Depth/km | Author | Catalog | Contributor | ContributorID | MagType | Magnitude | MagAuthor | EventLocationName
  #myId|#EventID | Time | Latitude | Longitude | Depth/km | Author | Catalog | Contributor | ContributorID | MagType | Magnitude | MagAuthor | EventLocationName
  E7|11512781|2022-01-03T09:46:35|23.9938|122.2595|19.0|us,usauto,pt|NEIC PDE|us|us7000g8n3,pt22003001,usauto7000g8n3|mww|6.2|us|TAIWAN REGION
  E6|11519353|2022-01-21T16:08:37|32.7437|132.0432|39.0|at,us,usauto,pt|NEIC PDE|us|us7000gdwz,pt22021000,at00r62i6f,usauto7000gdwz|Mww|6.3|us|SHIKOKU, JAPAN
  E5|11540112|2022-03-16T14:34:27|37.6468|141.6735|57.18|us,usauto,pt,at|NEIC PDE|us|usauto6000h518,us6000h518,pt22075003,at00r8udte|mwb|6.0|us|NEAR EAST COAST OF HONSHU, JAPAN
  E11|11414199|2021-05-13T23:58:14|37.7079|141.7784|32.0|us,usauto,pt,at|NEIC PDE|us|usauto7000e2xd,us7000e2xd,pt21134000,at00qt2l91|mww|6.0|us|NEAR EAST COAST OF HONSHU, JAPAN
  E4|11496103|2021-11-10T15:45:13|23.5934|126.4478|12.0|us,pt,usauto,at|NEIC PDE|us|us7000fszl,usauto7000fszl,pt21314000,at00r2d53g|Mww|6.6|us|SOUTHEAST OF RYUKYU ISLANDS
  E1|11505493|2021-12-09T02:05:07|29.4132|129.3846|7.0|us,usauto,pt,at|NEIC PDE|us|at00r3tsgo,usauto6000gaq5,pt21343000,us6000gaq5|Mww|6.0|us|RYUKYU ISLANDS, JAPAN
  E2|11350948|2020-12-10T13:19:58|24.7632|122.0098|73.17|us,usauto,pt,at|NEIC PDE|us|us7000cpqz,pt20345000,usauto7000cpqz,at00ql4l1c|mww|6.1|us|TAIWAN REGION
  E10|10993586|2019-01-08T12:39:31|30.5926|131.0371|35.0|us|NEIC PDE|us|us2000j1d4|Mww|6.3|us|KYUSHU, JAPAN
  E8|11024574|2019-04-11T08:18:21|40.4096|143.2985|18.0|us|NEIC PDE|us|us700034d3|Mww|6.0|us|OFF EAST COAST OF HONSHU, JAPAN
  E1|11091618|2019-08-04T10:23:03|37.7597|141.6089|38.0|us|NEIC PDE|us|us600050if|Mww|6.3|us|NEAR EAST COAST OF HONSHU, JAPAN
  E7|10639985|2018-02-06T15:50:43|24.1359|121.658|17.0|us|NEIC PDE|us|us1000chhc|Mww|6.4|us|TAIWAN
  E9|10608258|2018-01-24T10:51:19|41.1034|142.4323|31.0|us|NEIC PDE|us|us2000cnnl|Mww|6.3|us|HOKKAIDO, JAPAN REGION
  E6|10902101|2018-07-06T01:40:04|51.4994|157.8404|45.0|us|NEIC PDE|us|us2000fxyz|Mww|6.1|us|NEAR EAST COAST OF KAMCHATKA
  E3|10957286|2018-10-09T07:45:11|49.3941|156.2319|20.0|us|NEIC PDE|us|us1000h97j|Mww|6.1|us|KURIL ISLANDS
  E5|10936428|2018-08-16T18:22:53|23.4226|143.3187|20.0|us|NEIC PDE|us|us1000gaqv|Mww|6.3|us|VOLCANO ISLANDS, JAPAN REGION
  E4|10944513|2018-09-05T18:07:59|42.6861|141.9294|35.0|us,pt,at|NEIC PDE|us|pt18248000,at00pelgzo,us2000h8ty|Mww|6.6|us|HOKKAIDO, JAPAN REGION
  E3|10406737|2017-09-20T16:37:16|37.9814|144.6601|11.0|us|NEIC PDE|us|us2000arxv|Mww|6.1|us|OFF EAST COAST OF HONSHU, JAPAN
  E4|10401792|2017-09-07T17:26:49|27.7829|139.8041|451.0|us|NEIC PDE|us|us2000ahlq|Mww|6.1|us|BONIN ISLANDS, JAPAN REGION
  E2|10411586|2017-10-06T07:59:32|37.5033|144.0201|9.0|us|NEIC PDE|us|us2000b1v8|Mww|6.2|us|OFF EAST COAST OF HONSHU, JAPAN
  E1|10453666|2017-11-09T07:42:11|32.5208|141.438|12.0|us|NEIC PDE|us|us2000bkrv|Mww|6.0|us|SOUTHEAST OF HONSHU, JAPAN
  E9|5184131|2016-05-31T05:23:47|25.5615|122.5458|246.4|us,gcmt|NEIC PDE|us|gcmt20160531052347,us20005zay|mww|6.4|us|TAIWAN REGION
  E8|5189735|2016-08-04T16:24:33|24.9447|142.0141|510.0|us,gcmt|NEIC PDE||gcmt20160804162434,us10006a2k|mww|6.3|us,gcmt|VOLCANO ISLANDS, JAPAN REGION
  E3|5196026|2016-10-21T05:07:23|35.3676|133.8148|5.7|us,at,pt,gcmt|NEIC PDE|us|at00ofdswb,us20007fta,gcmt20161021050723,pt16295050|Mww|6.2|us|WESTERN HONSHU, JAPAN
  E2|5197585|2016-11-11T21:42:59|38.4973|141.5658|42.4|us|NEIC PDE|us|us1000770m|mww|6.1||NEAR EAST COAST OF HONSHU, JAPAN
  E15|10625663|2015-02-16T23:06:28|39.9468|143.1771|10.8|ISC|ISC|ISC|608402324|MW|6.7|GCMT|OFF EAST COAST OF HONSHU, JAPAN
  E14|5006299|2015-02-20T04:25:23|39.8189|143.6157|13.3|ISC|ISC|ISC|610586348|MW|6.2|GCMT|OFF EAST COAST OF HONSHU, JAPAN
  E10|5113626|2015-05-12T21:12:58|38.9005|142.0217|39.3|ISC|ISC|ISC|610588291|MW|6.8|GCMT|NEAR EAST COAST OF HONSHU, JAPAN
  E13|5006407|2015-02-21T10:13:53|39.8287|143.5175|12.2|ISC|ISC|ISC|610586373|MW|6.0|GCMT|OFF EAST COAST OF HONSHU, JAPAN
  E5|5150490|2015-07-07T05:10:27|43.9697|148.0042|51.1|ISC|ISC|ISC|611836168|MW|6.3|GCMT|EAST OF KURIL ISLANDS
  E7|10778316|2015-06-08T06:01:10|41.5071|142.0564|60.7|ISC|ISC|ISC|608295904|MW|6.1|GCMT|HOKKAIDO, JAPAN REGION
  E2|5161913|2015-10-14T05:43:08|48.8595|156.2259|12.0|us|NEIC PDE|us|us10003nib|mww|6.0|us|EAST OF KURIL ISLANDS
  E14|4479338|2014-03-13T17:06:51|33.6222|131.8077|83.4|ISC|ISC|ISC|604156129|MW|6.3|GCMT|KYUSHU, JAPAN
  E7|10360772|2014-08-10T03:43:19|41.2001|142.2065|55.6|IDC|ISC|ISC|610785780|MW|6.1|GCMT|HOKKAIDO, JAPAN REGION
  E15|4374929|2014-03-02T20:11:22|27.4238|127.3279|118.9|ISC|ISC|ISC|604087249|MW|6.5|GCMT|RYUKYU ISLANDS, JAPAN
  E6|4766096|2014-08-10T03:43:18|41.134|142.279|50.6|NIED|ISC|ISC|610785778|Mwc|6.1|GCMT|HOKKAIDO, JAPAN REGION
  E4|10530532|2014-11-09T14:38:15|46.93|140.63|10.0|SKHL|ISC|ISC|608910839|mb|7.6|SKHL|PRIMOR'YE, RUSSIA"""

  tempDataList = dataString.split("\n")[3:]

  dataList=[]
  for entry in tempDataList:

    temp = entry.split("|")
    temp2 = [temp[0].strip(),temp[2].split("-")[0], temp[3:6],temp[11]]  #[Name, year, lat, long, depth, mag]

    dataList.append(temp2)
    #print(temp2)

  return dataList


dataList = getOutputs()

dataList.reverse()   #so it's in the correct order as the loaded data

print(len(dataList))
dataValuesList=[]
for entry in dataList:
  #print(entry)

  temp = entry[2]

  temp.append(entry[3])

  temp = [float(k) for k in temp]

  dataValuesList.append(temp)

  print(dataValuesList[-1])




#Standardising the data:

from math import sqrt

N=len(dataValuesList)
Means=[0,0,0,0]
SDs=[0,0,0,0]
for event in dataValuesList:

  for i in range(4):
    Means[i]+=event[i]/N #mean
    SDs[i]+=(event[i]**2)/N #mean of the squares

for i in range(4):
  SDs[i] -= Means[i]**2 #- square of the means for the variance
  SDs[i] = sqrt(SDs[i]) #but rooted for the standard deviation

for n in range(N):
  event = dataValuesList[n]

  for i in range(4):
    event[i] = (event[i]-Means[i])/SDs[i]

  dataValuesList[n]=event



print(Means)
print(SDs)


# Means
#[35.9490611111111, 139.10234444444444, 63.99305555555556, 6.269444444444443]
#Variances
#[7.840629450223421, 9.138413709915895, 109.82974950701968, 0.30442974417067376]


36
[46.93, 140.63, 10.0, 7.6]
[41.134, 142.279, 50.6, 6.1]
[27.4238, 127.3279, 118.9, 6.5]
[41.2001, 142.2065, 55.6, 6.1]
[33.6222, 131.8077, 83.4, 6.3]
[48.8595, 156.2259, 12.0, 6.0]
[41.5071, 142.0564, 60.7, 6.1]
[43.9697, 148.0042, 51.1, 6.3]
[39.8287, 143.5175, 12.2, 6.0]
[38.9005, 142.0217, 39.3, 6.8]
[39.8189, 143.6157, 13.3, 6.2]
[39.9468, 143.1771, 10.8, 6.7]
[38.4973, 141.5658, 42.4, 6.1]
[35.3676, 133.8148, 5.7, 6.2]
[24.9447, 142.0141, 510.0, 6.3]
[25.5615, 122.5458, 246.4, 6.4]
[32.5208, 141.438, 12.0, 6.0]
[37.5033, 144.0201, 9.0, 6.2]
[27.7829, 139.8041, 451.0, 6.1]
[37.9814, 144.6601, 11.0, 6.1]
[42.6861, 141.9294, 35.0, 6.6]
[23.4226, 143.3187, 20.0, 6.3]
[49.3941, 156.2319, 20.0, 6.1]
[51.4994, 157.8404, 45.0, 6.1]
[41.1034, 142.4323, 31.0, 6.3]
[24.1359, 121.658, 17.0, 6.4]
[37.7597, 141.6089, 38.0, 6.3]
[40.4096, 143.2985, 18.0, 6.0]
[30.5926, 131.0371, 35.0, 6.3]
[24.7632, 122.0098, 73.17, 6.1]
[29.4132, 129.3846, 7.0, 6.0]
[23.5934, 126.4478, 12.0, 6.6]
[37.7079, 1

**load libraries for importing and formatting the input data:**
---



In [6]:
from obspy import read as readobs
from obspy import read_inventory
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pickle
import os
import random
import math

**Import the "original" data from mseed files, format them:**

In [7]:
# Read all datafiles in folder and store them in "events":

events = []
K=0
for year in years:
    print("\n=============")
    print(year)
    yearPath = './'+str(year)

    eventNames = os.listdir(yearPath)
    print(str(len(eventNames))+" events")
    print("=============\n")

    K = K+len(eventNames)
    for event in eventNames:
      #i+=1
      #print("Event" + str(i))
      print(event)

      #for i in range(numStations):
      events.append([readobs("./"+str(year)+"/"+event+"/"+station+".mseed") for station in stationNames])
      #events.append(read("./Data/seeddata32/TEST8.mseed"))

print(K)


# Form selectedevents, extract the 3 channels as data, and store them in "eventdata"
eventdata = []
for I in range(len(events)):
    event = events[I] #for each event's data
    stationData = []
    for station in event: # for each station
      flag = False
      for i in range(3): #for each axis (x,y,z), deals with multiple stations e.g for 2 stations gives a list of the form [x1,y1,z1,x2,y2,z2]
        stationData.append(station[i].data)
        #print(len(station[i].data))
        if len(station[i].data)!=1440000:
          print(len(station[i].data))
          flag=True
          print("Flagged!")
          print("Errors on "+str(I))
          #print("Errors on "+station)
          print("Errors on "+str(i))
          print("")
          K-=1
          break
    if flag ==False:
      eventdata.append(stationData)
print(K)
    
#Concatenate channels for each event
eventdataconcat = []
for stationAxis in eventdata:
    data_concat_TEMP=0
    for i in range(3*numStations):
      st_TEMP = np.expand_dims(stationAxis[i],axis=-1)

      if isinstance(data_concat_TEMP,int):
        data_concat_TEMP=st_TEMP
      else:
        data_concat_TEMP = np.c_[data_concat_TEMP,st_TEMP]

    #sta = np.expand_dims(event[0], axis=-1)
    #stb = np.expand_dims(event[1], axis=-1)
    #stc = np.expand_dims(event[2], axis=-1)
    eventdataconcat.append(data_concat_TEMP)

#print(eventdataconcat[0][0])



2014
5 events

2014_E4.996941
2014_E6.125507
2014_E15.958592
2014_E7.767260
2014_E14.417538

2015
6 events

2015_E2.41843
2015_E7.512251
2015_E5.34634
2015_E10.502950
2015_E14.799543
2015_E15.257925

2016
4 events

2016_E2.15144
2016_E3.823861
2016_E8.149332
2016_E9.871807

2017
4 events

2017_E1.727519
2017_E2.174189
2017_E4.721584
2017_E3.654490

2018
6 events

2018_E4.590218
2018_E5.447225
2018_E3.920499
2018_E6.658486
2018_E9.425817
2018_E7.37024

2019
3 events

2019_E1.463352
2019_E8.941687
2019_E10.518755

2020
1 events

2020_E2.774661

2021
3 events

2021_E1
2021_E4
2021_E11

2022
3 events

2022_E5
2022_E6
2022_E7
35
1153317
Flagged!
Errors on 20
Errors on 0

34


The structure of each event in "eventdataconcat" is (for 2 stations): <br>
array([  <br>
[x1, y1, z1, x2, y2, z2], # time 1 <br>
[x1, y1, z1, x2, y2, z2], # time 2 <br>
..... <br>
[x1, y1, z1, x2, y2, z2]  # time n <br>
]) <br>
(n is the total number of time samples)

### Extract first and last 40k samples and class as noise and precusros, respectivley

In [8]:
def create_precursors(X1, last = 40000):
    precurstr = X1[-last:] 
    return(precurstr)
    
#def create_noise(X1, first = 0, second = 40000):
#    precurstr = X1[first:second]
       
#    return(precurstr)

#Select the final 40000 time steps from each event (all 3 channels) and label as 'precursors' 
#Select the first 40000 time steps from each event (all 3 channels) and label as 'noise' 

precursors = []
#noise = []
for event in eventdataconcat:
    precursors.append(create_precursors(event,40000))
    
#for event in eventdataconcat:
#    noise.append(create_noise(event, 0, 40000))

### Make windows of length "window_length" from the selected 'noise' and 'precursor' data 
The windows overlap of 650 timesteps
With this overlap, each precursor window of 40000 samples produces 37 "normpre"v windows of 16384 samples. 
Same for noise windows.

In [9]:
window_length=16384

def make_windows(X1, sample_stride = 650):
    X2 = []
    for i in range(len(X1)-window_length):        
        if i % sample_stride == 0:
               X2.append(X1[i:i+window_length])    
    return(X2)
   
precursor_windows = []
for event in precursors:
    precursor_windows.append(make_windows(event, 650))
    
#noise_windows = []
#for event in noise:
#    noise_windows.append(make_windows(event, 650))

### Normalisation of signal
Three different options for normalisation fuction: 
- new 
- new2
- old

In previous work (Ong et al.) normalisation_old was used. 

In [10]:
def normalise_new(X1): # Here each 3 individual component of each window is mean stripped and normalised
    X2 = []
    for data in X1:
        values = np.zeros((len(data),3))
        mea=np.mean(data,axis=0).reshape(-1, 1)
        values = (data.T-mea).T
        values = values / np.linalg.norm(values,axis=0,ord=np.Inf) # L_infinity norm???????????
        X2.append(values)
    return X2

def normalise_new2(X1): # not normalise, not strip mean
    X2 = []
    for data in X1:
        values = np.zeros((len(data),3))
        values = data
        X2.append(values)
    return X2

def normalise_old(X1): ## Here the 3 components are mean stripped and normalised together, results strange but working for the CNN:
    X2 = []
    for data in X1:
        values = np.zeros((len(data),3))
        values = data - np.mean(data)
        values = values / np.linalg.norm(values)
        X2.append(values)
    return X2


normpre = []
normnoise = []

if choice == 'new2':
  for event in precursor_windows:    normpre.append(normalise_new2(event))
  #for event in noise_windows:    normnoise.append(normalise_new2(event))
elif choice == 'old':
  for event in precursor_windows:    normpre.append(normalise_old(event))
  #for event in noise_windows:    normnoise.append(normalise_old(event))
elif choice == 'new':
  for event in precursor_windows:    normpre.append(normalise_new(event))
  #for event in noise_windows:    normnoise.append(normalise_new(event))


### Select events for train and test datasets. These events were randomly selected.

In [11]:
trainlist = []

trainlist = [1, 3, 4, 5, 6, 7, 9, 10, 11, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
testlist = [0, 2, 8, 12, 15, 19, 33]

if len(trainlist)==0:

  K=len(eventdataconcat)

  print(K)

  trainlist = random.sample(range(K),round(4*K/5))

  temp = set(trainlist)
  temp2 = set(range(K))

  testlist = list(temp2-temp)

print(sorted(trainlist))
print(len(trainlist))
print()
print(sorted(testlist))
print(len(testlist))






[1, 3, 4, 5, 6, 7, 9, 10, 11, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
27

[0, 2, 8, 12, 15, 19, 33]
7


## output data to files for further use by the CNN

In [12]:
print(choice)
pickle.dump( normpre, open( "./Feature_Data/normpre_"+choice+".p", "wb" ) ) # pickle file with all precursor windows
pickle.dump( dataValuesList, open( "./Feature_Data/normnoise_"+choice+".p", "wb" ) ) # pickle file with all noise windows
pickle.dump( trainlist, open( "./Feature_Data/trainlist.p", "wb" ) ) # pickel file with the labels of events in train data
pickle.dump( testlist, open( "./Feature_Data/testlist.p", "wb" ) ) # pickle file with labels of events in test data


new2
